# Weekly ML Code Club - 001
## Data Wrangling
Wrangle some data from Gapminder, a Swedish non-profit co-founded by Hans Rosling. "Gapminder produces free teaching resources making the world understandable based on reliable statistics."

## Datasets Used
https://raw.githubusercontent.com/zalihat/cell_phone_dataset/master/cell_phones.csv
https://raw.githubusercontent.com/zalihat/cell_phone_dataset/master/population1.csv
https://github.com/open-numbers/ddf--gapminder--systema_globalis/blob/master/ddf--entities--geo--country.csv

In [113]:
# Import necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import json

## Part One - Load the data

In [114]:
# Read cell phone dataset
url='https://raw.githubusercontent.com/zalihat/cell_phone_dataset/master/cell_phones.csv'
df_cell = pd.read_csv(url)
df_cell.head()

,Country,year,cell_phones_total
0,Afghanistan,1960,0.0
1,Afghanistan,1961,0.0
2,Afghanistan,1962,0.0
3,Afghanistan,1963,0.0
4,Afghanistan,1964,0.0


In [115]:
# Read population dataset
url1='https://raw.githubusercontent.com/zalihat/cell_phone_dataset/master/population1.csv'
df_pop = pd.read_csv(url1)
df_pop.head()

,Country,year,population
0,Afghanistan,1800,3280000
1,Afghanistan,1801,3280000
2,Afghanistan,1802,3280000
3,Afghanistan,1803,3280000
4,Afghanistan,1804,3280000


In [116]:
# Read geo-country dataset
url2 = 'https://github.com/open-numbers/ddf--gapminder--systema_globalis/blob/master/ddf--entities--geo--country.csv'
def html_to_df(url, clean_empty = False , attrs = {}, helper = None):
    
    html_content = requests.get(url).text
    soup = bs(html_content, "lxml")
    table = str(soup.find("table", attrs=attrs))
    df = pd.read_html(str(table))[0]
    
    if clean_empty :
        df = df.loc[:, ~df.isnull().all(axis = 0)]
        
    if helper:
        df = helper(df)   
    return df

In [117]:
df_geo = html_to_df(url2,clean_empty=True)
df_geo.head()

,country,g77_and_oecd_countries,income_3groups,income_groups,is--country,iso3166_1_alpha2,iso3166_1_alpha3,iso3166_1_numeric,iso3166_2,landlocked,...,longitude,main_religion_2008,name,un_sdg_ldc,un_sdg_region,un_state,unicef_region,unicode_region_subtag,world_4region,world_6region
0,abkh,others,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Abkhazia,NaN,NaN,False,NaN,NaN,europe,europe_central_asia
1,abw,others,high_income,high_income,True,AW,ABW,533.0,NaN,coastline,...,-69.96667,christian,Aruba,un_not_least_developed,un_latin_america_and_the_caribbean,False,NaN,AW,americas,america
2,afg,g77,low_income,low_income,True,AF,AFG,4.0,NaN,landlocked,...,66.00000,muslim,Afghanistan,un_least_developed,un_central_and_southern_asia,True,sa,AF,asia,south_asia
3,ago,g77,middle_income,lower_middle_income,True,AO,AGO,24.0,NaN,coastline,...,18.50000,christian,Angola,un_least_developed,un_sub_saharan_africa,True,ssa,AO,africa,sub_saharan_africa
4,aia,others,NaN,NaN,True,AI,AIA,660.0,NaN,coastline,...,-63.05000,christian,Anguilla,un_not_least_developed,un_latin_america_and_the_caribbean,False,NaN,AI,americas,america


## Part Two - Join cellphone and Population Dataframe

In [118]:
df1=df_cell.merge(df_pop, how='inner', on=['Country','year'])
df1.head()

,Country,year,cell_phones_total,population
0,Afghanistan,1960,0.0,9000000
1,Afghanistan,1961,0.0,9170000
2,Afghanistan,1962,0.0,9350000
3,Afghanistan,1963,0.0,9540000
4,Afghanistan,1964,0.0,9740000


In [119]:
df1.shape

(11640, 4)

In [120]:
df_geo.rename(columns={'country':'geo','name':'Country'},inplace=True)
df_geo.head()

,geo,g77_and_oecd_countries,income_3groups,income_groups,is--country,iso3166_1_alpha2,iso3166_1_alpha3,iso3166_1_numeric,iso3166_2,landlocked,...,longitude,main_religion_2008,Country,un_sdg_ldc,un_sdg_region,un_state,unicef_region,unicode_region_subtag,world_4region,world_6region
0,abkh,others,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Abkhazia,NaN,NaN,False,NaN,NaN,europe,europe_central_asia
1,abw,others,high_income,high_income,True,AW,ABW,533.0,NaN,coastline,...,-69.96667,christian,Aruba,un_not_least_developed,un_latin_america_and_the_caribbean,False,NaN,AW,americas,america
2,afg,g77,low_income,low_income,True,AF,AFG,4.0,NaN,landlocked,...,66.00000,muslim,Afghanistan,un_least_developed,un_central_and_southern_asia,True,sa,AF,asia,south_asia
3,ago,g77,middle_income,lower_middle_income,True,AO,AGO,24.0,NaN,coastline,...,18.50000,christian,Angola,un_least_developed,un_sub_saharan_africa,True,ssa,AO,africa,sub_saharan_africa
4,aia,others,NaN,NaN,True,AI,AIA,660.0,NaN,coastline,...,-63.05000,christian,Anguilla,un_not_least_developed,un_latin_america_and_the_caribbean,False,NaN,AI,americas,america


In [121]:
df2=merge_cell_pop.merge(df_geo[['geo','Country']],how='inner',on='Country')
df2.head()

,Country,year,cell_phones_total,population,geo
0,Afghanistan,1960,0.0,9000000,afg
1,Afghanistan,1961,0.0,9170000,afg
2,Afghanistan,1962,0.0,9350000,afg
3,Afghanistan,1963,0.0,9540000,afg
4,Afghanistan,1964,0.0,9740000,afg


In [122]:
df2.shape

(11640, 5)

## Part Three - Make Features

In [123]:
df2['No_of_cell_phones_per_person'] = df2['cell_phones_total']/df2['population']
df2.head()

,Country,year,cell_phones_total,population,geo,No_of_cell_phones_per_person
0,Afghanistan,1960,0.0,9000000,afg,0.0
1,Afghanistan,1961,0.0,9170000,afg,0.0
2,Afghanistan,1962,0.0,9350000,afg,0.0
3,Afghanistan,1963,0.0,9540000,afg,0.0
4,Afghanistan,1964,0.0,9740000,afg,0.0


In [124]:
df2.loc[(df2['Country'] == 'United States') & (df2['year'] == 2017)]

,Country,year,cell_phones_total,population,geo,No_of_cell_phones_per_person
11157,United States,2017,400000000.0,325000000,usa,1.230769


In [125]:
df2['geo']=df2['geo'].str.upper()
df2.head()

,Country,year,cell_phones_total,population,geo,No_of_cell_phones_per_person
0,Afghanistan,1960,0.0,9000000,AFG,0.0
1,Afghanistan,1961,0.0,9170000,AFG,0.0
2,Afghanistan,1962,0.0,9350000,AFG,0.0
3,Afghanistan,1963,0.0,9540000,AFG,0.0
4,Afghanistan,1964,0.0,9740000,AFG,0.0


## Part Four - Process Data

In [126]:
# Descriptive statistics of numeric columns
df2.describe()

,year,cell_phones_total,population,No_of_cell_phones_per_person
count,11640.000000,1.164000e+04,1.164000e+04,11640.000000
mean,1989.500000,7.875722e+06,2.710307e+07,0.235536
std,17.318846,5.572496e+07,1.086187e+08,0.433303
min,1960.000000,0.000000e+00,4.380000e+03,0.000000
25%,1974.750000,0.000000e+00,1.117500e+06,0.000000
50%,1989.500000,0.000000e+00,5.100000e+06,0.000000
75%,2004.250000,5.615000e+05,1.590000e+07,0.233667
max,2019.000000,1.730000e+09,1.430000e+09,2.126068


In [127]:
# Statistics of string columns
df2.describe(include=[object])

,Country,geo
count,11640,11640
unique,194,194
top,Bahamas,LBR
freq,60,60


### In 2017, what were the top 5 countries with the most cell phones total?

In [128]:
df3=df2.loc[df2['year'] == 2017]

In [129]:
df4=df3[['Country','cell_phones_total']].sort_values(by=['cell_phones_total'],ascending=False).head()
df4

,Country,cell_phones_total
2157,China,1.470000e+09
4617,India,1.170000e+09
4677,Indonesia,4.350000e+08
11157,United States,4.000000e+08
8577,Russia,2.270000e+08
